In [2]:
import pandas as pd
import sklearn as skr
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta

In [3]:
# Load the balance data
def load_data(path='user_balance_table.csv'):
    data_balance = pd.read_csv(path)
    data_balance = add_timestamp(data_balance)
    return data_balance.reset_index(drop=True)
    

# add tiemstamp to dataset
def add_timestamp(data, time_index='report_date'):
    data_balance = data.copy()
    data_balance['date'] = pd.to_datetime(data_balance[time_index],
                                          format= "%Y%m%d")
    data_balance['day'] = data_balance['date'].dt.day
    data_balance['month'] = data_balance['date'].dt.month
    data_balance['year'] = data_balance['date'].dt.year
    data_balance['week'] = data_balance['date'].dt.week
    data_balance['weekday'] = data_balance['date'].dt.weekday
    return data_balance.reset_index(drop=True)

# total amount
def get_total_balance(data,date):
    df_tmp = data.copy()
    df_tmp = df_tmp.groupby(['date'])['total_purchase_amt',
                                      'total_redeem_amt'].sum()
    df_tmp.reset_index(inplace=True)
    return df_tmp[(df_tmp['date']>= date)].reset_index(drop=True)

# Generate the test data
def generate_test_data(data):
    total_balance = data.copy()
    start = datetime.datetime(2014,9,1)
    testdata = []
    while start != datetime.datetime(2014,10,15):
        temp = [start, np.nan, np.nan]
        testdata.append(temp)
        start += datetime.timedelta(days = 1)
    testdata = pd.DataFrame(testdata)
    testdata.columns = total_balance.columns

    total_balance = pd.concat([total_balance, testdata], axis = 0)
    total_balance = total_balance.reset_index(drop=True)
    return total_balance.reset_index(drop=True)

# Load user's information
def load_user_information(path='user_profile_table.csv'):
    return pd.read_csv(path)

In [4]:
# 载入数据
import os
os.chdir('D:/Dataset/PRD')
balance_data = load_data('user_balance_table.csv')
balance_data = add_timestamp(balance_data)
total_balance = get_total_balance(balance_data, date = '2014-03-01')
total_balance = generate_test_data(total_balance)
total_balance = add_timestamp(total_balance, 'date')

In [5]:
total_balance.head()

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday
0,2014-03-01,362865580.0,211279011.0,1,3,2014,9,5
1,2014-03-02,276202230.0,246199417.0,2,3,2014,9,6
2,2014-03-03,505305862.0,513017360.0,3,3,2014,10,0
3,2014-03-04,524146340.0,250562978.0,4,3,2014,10,1
4,2014-03-05,454295491.0,209072753.0,5,3,2014,10,2


In [39]:
# 创建数据的深层拷贝

data = total_balance.copy()

In [45]:
# 定义生成时间序列规则预测结果的方法
def generate_base(df, month_index):
    # 选中固定时间段的数据集
    total_balance = df.copy()
    total_balance = total_balance[['date','total_purchase_amt',
                                   'total_redeem_amt']]
    total_balance = total_balance[
        (total_balance['date'] >= pd.Timestamp(
            datetime.date(2014,3,1))) & (total_balance['date'] < pd.Timestamp(datetime.date(2014, month_index, 1)))]
    # 加入时间戳
    total_balance['weekday'] = total_balance['date'].dt.weekday
    total_balance['day'] = total_balance['date'].dt.day
    total_balance['week'] = total_balance['date'].dt.week
    total_balance['month'] = total_balance['date'].dt.month
    
    # 统计翌日因子
    mean_of_each_weekday = total_balance[
        ['weekday']+['total_purchase_amt','total_redeem_amt']
    ].groupby('weekday',as_index=False).mean()
    for name in ['total_purchase_amt','total_redeem_amt']:
        mean_of_each_weekday = mean_of_each_weekday.rename(
            columns={name: name+'_weekdaymean'})
    mean_of_each_weekday['total_purchase_amt_weekdaymean'] /= np.mean(
        total_balance['total_purchase_amt'])
    mean_of_each_weekday['total_redeem_amt_weekdaymean'] /= np.mean(
        total_balance['total_redeem_amt'])

    # 合并统计结果到原数据集
    total_balance = pd.merge(total_balance, 
                             mean_of_each_weekday, on='weekday', how='left')

    # 分别统计翌日在(1~31)号出现的频次
    weekday_count = total_balance[['day','weekday','date']
                                 ].groupby(['day','weekday'],as_index=False).count()
    weekday_count = pd.merge(weekday_count, 
                             mean_of_each_weekday, on='weekday')

    # 依据频次对翌日因子进行加权，获得日期因子
    weekday_count['total_purchase_amt_weekdaymean'] *= weekday_count[
        'date']   / len(np.unique(total_balance['month']))
    weekday_count['total_redeem_amt_weekdaymean'] *= weekday_count[
        'date']  / len(np.unique(total_balance['month']))
    day_rate = weekday_count.drop(
        ['weekday','date'],axis=1).groupby('day',as_index=False).sum()

    # 将训练集中所有日期的均值剔除日期残差得到base
    day_mean = total_balance[
        ['day'] + ['total_purchase_amt','total_redeem_amt']
                         ].groupby('day',as_index=False).mean()
    day_pre = pd.merge(day_mean, day_rate, on='day', how='left')
    day_pre['total_purchase_amt'] /= day_pre[
        'total_purchase_amt_weekdaymean']
    day_pre['total_redeem_amt'] /= day_pre['total_redeem_amt_weekdaymean']

    # 生成测试集数据
    for index, row in day_pre.iterrows():
        if month_index in (2,4,6,9) and row['day'] == 31:
            break
        day_pre.loc[index, 'date'] = datetime.datetime(
            2014, month_index, int(row['day']))

    # 基于base与翌日因子获得最后的预测结果
    day_pre['weekday'] = day_pre.date.dt.weekday
    day_pre = day_pre[['date','weekday']+[
        'total_purchase_amt','total_redeem_amt']]
    day_pre = pd.merge(day_pre, mean_of_each_weekday,on='weekday')
    day_pre['total_purchase_amt'] *= day_pre[
        'total_purchase_amt_weekdaymean']
    day_pre['total_redeem_amt'] *= day_pre[
        'total_redeem_amt_weekdaymean']

    day_pre = day_pre.sort_values(
        'date')[['date']+['total_purchase_amt','total_redeem_amt']]
    return day_pre

In [46]:
# 生成预测结果（以及残差）

base_list = []
for i in range(4, 10):
    base_list.append(generate_base(data, i).reset_index(drop=True))

In [48]:
base = pd.concat(base_list).reset_index(drop=True)
for i in ['total_purchase_amt','total_redeem_amt']:
    base = base.rename(columns={i: i+'_base'})

data = pd.merge(data.reset_index(drop=True), 
                base.reset_index(drop=True), 
                on='date', how='left').reset_index(drop=True)

In [51]:
b = data.loc[data['date']>='2014-04',:]

In [54]:
data['purchase_residual'] = data[
    'total_purchase_amt'] / data['total_purchase_amt_base']

data['redeem_residual'] = data[
    'total_redeem_amt'] / data['total_redeem_amt_base']

In [55]:
data.loc[data['date']>='2014-04',:].head()

,date,total_purchase_amt,total_redeem_amt,day,month,year,week,weekday,total_purchase_amt_base,total_redeem_amt_base,purchase_residual,redeem_residual
31,2014-04-01,453320585.0,277429358.0,1,4,2014,14,1,6.207296e+08,4.373304e+08,0.730303,0.634370
32,2014-04-02,355347118.0,272612066.0,2,4,2014,14,2,4.342629e+08,3.578884e+08,0.818276,0.761724
33,2014-04-03,363877120.0,266605457.0,3,4,2014,14,3,4.734257e+08,3.791019e+08,0.768604,0.703255
34,2014-04-04,251895894.0,200192637.0,4,4,2014,14,4,3.703544e+08,2.161569e+08,0.680148,0.926145
35,2014-04-05,202336542.0,163199682.0,5,4,2014,14,5,3.093513e+08,9.861873e+07,0.654067,1.654855


In [56]:
# 对结果表重命名

data = data[
    ['date','purchase_residual','redeem_residual',
     'total_purchase_amt_base', 'total_redeem_amt_base']]
for i in data.columns:
    if i == 'date':
        data[i] = data[i].astype(str)
        data[i] = data[i].str.replace('-','')
data.columns = [
    ['date'] + ['total_purchase_amt','total_redeem_amt'] + [
        'total_purchase_predicted_by_cycle',
        'total_redeem_predicted_by_cycle'] ]

In [33]:
# 保存预测结果到本地

data.to_csv('base.csv',index=False)

In [57]:
data.head()

,date,total_purchase_amt,total_redeem_amt,total_purchase_predicted_by_cycle,total_redeem_predicted_by_cycle
0,20140301,NaN,NaN,NaN,NaN
1,20140302,NaN,NaN,NaN,NaN
2,20140303,NaN,NaN,NaN,NaN
3,20140304,NaN,NaN,NaN,NaN
4,20140305,NaN,NaN,NaN,NaN
